#Genetic Algorithm
##Steps Of Genetic Algorithm

1. Encoding
2. Initial population
3. Fitness function
4. Selection
5. Mating pool
6. Crossover
7. Mutation



IMPORTING LIBRARIES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim

POPULATION GENERATION

In [ ]:
def gen_population(chromosome_length, population_size):
  population=[]
  for i in range(population_size):
    chromosome = random.choices([0, 1], k=chromosome_length)
    population.append(chromosome)

  return population

DECODING THE TILE SIZE

In [ ]:
def bin2dec(chromosome):
  decimal=0

  for i in chromosome:
    decimal=decimal*2 + int(i)

  return decimal

DIVIDING THE CHROMOSOMES FOR LENGTH AND BREADTH

In [ ]:
def divide_chromosome(chromosome):
  mbits=len(chromosome)//2
  nbits=len(chromosome)-mbits

  m=chromosome[:mbits]
  n=chromosome[nbits:]

  m=bin2dec(m)
  n=bin2dec(n)

  return m,n

DEFINING A FITNESS FUNCTION

In [ ]:
def calculate_fitness(original_image,enhanced_image):
  org_img=np.asarray(original_image)
  enh_img=np.asarray(enhanced_image)


  mse=np.mean((org_img - enh_img) ** 2)
  maxpixel=255.0

  psnr=20*np.log10(maxpixel/np.sqrt(mse))

  return psnr

APPLYING CLAHE FOR FITNESS FUNCTION

In [ ]:
def ApplyClahe(image,cliplimit=2.0,gridsize=(8,8)):
  LABimage=cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
  L,A,B=cv2.split(LABimage)
  CLAHE=cv2.createCLAHE(cliplimit,gridsize)
  Enh_L=CLAHE.apply(L)
  Enh_LAB=cv2.merge((Enh_L,A,B))

  return cv2.cvtColor(Enh_LAB,cv2.COLOR_LAB2BGR)

FITNESS GENERATION

In [ ]:
def generate_fitness(population,image,eachgentable):
  Fitness_table=np.zeros(len(population))
  print("Length Of Chromosome: ",len(population[0]))
  for i, current_chromosome in enumerate(population):
    m,n=divide_chromosome(current_chromosome)
    grid=(m,n)
    Enh_image=ApplyClahe(image,gridsize=grid)
    Fitness_table[i]=calculate_fitness(image,Enh_image)
  eachgentable.append(Fitness_table)
  plt.plot(Fitness_table)
  return Fitness_table,eachgentable

SORTING FITNESS

In [ ]:
def sortfitness(population,fitness_table,bestfitnesstable):
    combined_arrays = [(population[i], fitness_table[i]) for i in range(len(population))]


    sorted_combined_arrays = sorted(combined_arrays, key=lambda x: x[1])

    sorted_population = [t[0] for t in sorted_combined_arrays]
    sorted_fitness_table = [t[1] for t in sorted_combined_arrays]

    # print("Fittest Chromosome's Score: ", sorted_fitness_table[len(population)-1], " Fittest Chromosome: ",sorted_population[len(population)-1])
    bestfitnesstable.append(sorted_fitness_table[len(population)-1])
    return sorted_population,sorted_fitness_table,bestfitnesstable


SELECTION

In [ ]:
def roulette_wheel_selection(fitness_table, population):
  fitness_cdf = np.cumsum(fitness_table)
  fitness_cdf = fitness_cdf / fitness_cdf[-1]

  rand_num = np.random.rand()
  for i in range(len(population)):
    if rand_num < fitness_cdf[i]:
      selected_parent = population[i]

      return selected_parent

TWO POINT CROSSOVER

In [ ]:
def crossover(parent1,parent2):
  point1,point2=np.random.choice(len(parent1),size=2,replace=False)
  child1=parent1[:min(point1,point2)]+parent2[min(point1,point2):max(point1,point2)]+parent1[max(point1,point2):]
  child2=parent2[:min(point1,point2)]+parent1[min(point1,point2):max(point1,point2)]+parent2[max(point1,point2):]
  return child1, child2


GENERATING NEW POPULATION

In [ ]:
def newgeneration(child1,child2,population):
  population[0]=child1
  population[1]=child2
  return population



THREE POINT MUTATION

In [ ]:
def mutation(population):
  selected_chromosome=random.randint(0,len(population)-2)

  point1=np.random.randint(0,len(population[0])-1)
  point2=np.random.randint(0,len(population[0])-1)
  point3=np.random.randint(0,len(population[0])-1)
  if population[selected_chromosome][point1]==1:
    population[selected_chromosome][point1]=0
  else:
    population[selected_chromosome][point1]=1

  if population[selected_chromosome][point2]==1:
    population[selected_chromosome][point2]=0
  else:
    population[selected_chromosome][point2]=1

  if population[selected_chromosome][point3]==1:
    population[selected_chromosome][point3]=0
  else:
    population[selected_chromosome][point3]=1

  return population


CODE FOR GA IMPLEMENTATION

In [ ]:
def GA_for_CLAHE(image,max_m,max_n,population_size,generations):
  chromosome_length=len(bin(max(max_m,max_n))[2:])*2
  population=gen_population(chromosome_length,population_size)
  Best_fitness=[]
  eachgentable=[]

  print("--------------------------------------------------------------------")

  for i in range(generations):
    print("Generation: ", i+1)
    fitness_table,eachgentable=generate_fitness(population,image,eachgentable)
    population,fitness_table,Best_fitness=sortfitness(population,fitness_table,Best_fitness)

    parent1=roulette_wheel_selection(fitness_table,population)
    parent2=roulette_wheel_selection(fitness_table,population)
    child1,child2=crossover(parent1,parent2)
    population=newgeneration(child1,child2,population)

    parent1=roulette_wheel_selection(fitness_table,population)
    parent2=roulette_wheel_selection(fitness_table,population)
    child1,child2=crossover(parent1,parent2)
    population=newgeneration(child1,child2,population)

    parent1=roulette_wheel_selection(fitness_table,population)
    parent2=roulette_wheel_selection(fitness_table,population)
    child1,child2=crossover(parent1,parent2)
    population=newgeneration(child1,child2,population)

    population=mutation(population)
    population=mutation(population)
    population=mutation(population)
    population=mutation(population)


    print("------------------------------------------------------------------")

  enhanced_image=applyclahe(image,population=population,population_size=population_size,max_m=max_m,max_n=max_n)


  return enhanced_image, Best_fitness, eachgentable

FUNCTION FOR APPLYING CLAHE FOR THE FINAL VALUES

In [ ]:
def applyclahe(image,population,population_size,max_m,max_n):
  best_chromosome = population[population_size-1]
  m,n = divide_chromosome(best_chromosome)

  m=max(min(m,max_m), 2)
  n=max(min(n,max_n), 2)

  print(m)
  print(n)

  grid=(m,n)
  enhanced_image=ApplyClahe(image,gridsize=grid)
  return enhanced_image

COMPARING IMAGES BY PLOTTING THEM

In [ ]:
def compare_images(original_image,enhanced_image):
  original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
  enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)

# Create a figure and axis objects
  fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Display original image
  axes[0].imshow(original_image)
  axes[0].set_title('Original')  # Add label 'Before' to the original image
  axes[0].axis('off')

# Display enhanced image
  axes[1].imshow(enhanced_image)
  axes[1].set_title('CLAHE')  # Add label 'After' to the enhanced image
  axes[1].axis('off')

# Show the plot
plt.show()

#TESTING THE MODEL

READING THE IMAGE

In [ ]:
img=cv2.imread("/content/image_30.jpeg")
plt.imshow(img)

RUNNING THE GA......

In [ ]:
image=img.copy()


max_m=image.shape[0]//4
max_n=image.shape[1]//4

min_m=2
min_n=2

population_size= 20
generations = 12

enhanced_image,Best_fitness,eachgentable=GA_for_CLAHE(image,max_m,max_n,population_size,generations)

print("Completed")

PLOTTING GRAPHS

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(Best_fitness)
plt.title('Best Fitness For Each Generation')
plt.xlabel('Generations')
plt.ylabel('Fitness Score')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(eachgentable[0])
plt.title('Fitness For First Generation')
plt.xlabel('Chromosome')
plt.ylabel('Fitness Score')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(eachgentable[4])
plt.title('Best Fitness After Half OF The Generation')
plt.xlabel('Chromosome')
plt.ylabel('Fitness Score')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(eachgentable[9])
plt.title('Best Fitness For 10th Generation')
plt.xlabel('Chromosome')
plt.ylabel('Fitness Score')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
compare_images(image,enhanced_image)

CALCULATING SSIM SCORE FOR THE IMAGES

In [ ]:
image2=image.copy()
LABimage2=cv2.cvtColor(image2,cv2.COLOR_BGR2LAB)
orgL,orgA,orgB=cv2.split(LABimage2)

image3=enhanced_image.copy()
LABimage3=cv2.cvtColor(image3,cv2.COLOR_BGR2LAB)
enhL,enhA,enhB=cv2.split(LABimage3)

score,_=ssim(orgL,enhL,full=True)

In [ ]:
score

In [ ]:
calculate_fitness(image,enhanced_image)